https://hyperledger-indy.readthedocs.io/projects/node/en/latest/transactions.html

https://github.com/hyperledger/indy-node/blob/master/docs/source/auth_rules.md


# 1. 필요한 모듈 불러오기

In [20]:
import time
from indy import anoncreds, did, ledger, pool, wallet, blob_storage
import json
import logging
from os.path import dirname
from indy.error import ErrorCode, IndyError
import asyncio
import json
from os import environ
from pathlib import Path
from tempfile import gettempdir

In [40]:
PROTOCOL_VERSION = 2

# 2. 유틸함수 정의

In [21]:
def path_home() -> Path:
    return Path.home().joinpath(".indy_client")

In [22]:
def get_pool_genesis_txn_path(pool_name):
    path_temp = Path(gettempdir()).joinpath("indy")
    path = path_temp.joinpath("{}.txn".format(pool_name))
    save_pool_genesis_txn_file(path)
    return path

In [23]:
def pool_genesis_txn_data():
    pool_ip = environ.get("TEST_POOL_IP", "127.0.0.1")

    return "\n".join([
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node1","blskey":"4N8aUNHSgjQVgkpm8nhNEfDf6txHznoYREg9kirmJrkivgL4oSEimFF6nsQ6M41QvhM2Z33nves5vfSn9n1UwNFJBYtWVnHYMATn76vLuL3zU88KyeAYcHfsih3He6UHcXDxcaecHVz6jhCYz1P2UZn2bDVruL5wXpehgBfBaLKm3Ba","blskey_pop":"RahHYiCvoNCtPTrVtP7nMC5eTYrsUA8WjXbdhNc8debh1agE9bGiJxWBXYNFbnJXoXhWFMvyqhqhRoq737YQemH5ik9oL7R4NTTCz2LEZhkgLJzB3QRQqJyBNyv7acbdHrAT8nQ9UkLbaVL9NBpnWXBTw4LEMePaSHEw66RzPNdAX1","client_ip":"{}","client_port":9702,"node_ip":"{}","node_port":9701,"services":["VALIDATOR"]}},"dest":"Gw6pDLhcBcoQesN72qfotTgFa7cbuqZpkX3Xo6pLhPhv"}},"metadata":{{"from":"Th7MpTaRZVRYnPiabds81Y"}},"type":"0"}},"txnMetadata":{{"seqNo":1,"txnId":"fea82e10e894419fe2bea7d96296a6d46f50f93f9eeda954ec461b2ed2950b62"}},"ver":"1"}}'.format(
            pool_ip, pool_ip),
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node2","blskey":"37rAPpXVoxzKhz7d9gkUe52XuXryuLXoM6P6LbWDB7LSbG62Lsb33sfG7zqS8TK1MXwuCHj1FKNzVpsnafmqLG1vXN88rt38mNFs9TENzm4QHdBzsvCuoBnPH7rpYYDo9DZNJePaDvRvqJKByCabubJz3XXKbEeshzpz4Ma5QYpJqjk","blskey_pop":"Qr658mWZ2YC8JXGXwMDQTzuZCWF7NK9EwxphGmcBvCh6ybUuLxbG65nsX4JvD4SPNtkJ2w9ug1yLTj6fgmuDg41TgECXjLCij3RMsV8CwewBVgVN67wsA45DFWvqvLtu4rjNnE9JbdFTc1Z4WCPA3Xan44K1HoHAq9EVeaRYs8zoF5","client_ip":"{}","client_port":9704,"node_ip":"{}","node_port":9703,"services":["VALIDATOR"]}},"dest":"8ECVSk179mjsjKRLWiQtssMLgp6EPhWXtaYyStWPSGAb"}},"metadata":{{"from":"EbP4aYNeTHL6q385GuVpRV"}},"type":"0"}},"txnMetadata":{{"seqNo":2,"txnId":"1ac8aece2a18ced660fef8694b61aac3af08ba875ce3026a160acbc3a3af35fc"}},"ver":"1"}}'.format(
            pool_ip, pool_ip),
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node3","blskey":"3WFpdbg7C5cnLYZwFZevJqhubkFALBfCBBok15GdrKMUhUjGsk3jV6QKj6MZgEubF7oqCafxNdkm7eswgA4sdKTRc82tLGzZBd6vNqU8dupzup6uYUf32KTHTPQbuUM8Yk4QFXjEf2Usu2TJcNkdgpyeUSX42u5LqdDDpNSWUK5deC5","blskey_pop":"QwDeb2CkNSx6r8QC8vGQK3GRv7Yndn84TGNijX8YXHPiagXajyfTjoR87rXUu4G4QLk2cF8NNyqWiYMus1623dELWwx57rLCFqGh7N4ZRbGDRP4fnVcaKg1BcUxQ866Ven4gw8y4N56S5HzxXNBZtLYmhGHvDtk6PFkFwCvxYrNYjh","client_ip":"{}","client_port":9706,"node_ip":"{}","node_port":9705,"services":["VALIDATOR"]}},"dest":"DKVxG2fXXTU8yT5N7hGEbXB3dfdAnYv1JczDUHpmDxya"}},"metadata":{{"from":"4cU41vWW82ArfxJxHkzXPG"}},"type":"0"}},"txnMetadata":{{"seqNo":3,"txnId":"7e9f355dffa78ed24668f0e0e369fd8c224076571c51e2ea8be5f26479edebe4"}},"ver":"1"}}'.format(
            pool_ip, pool_ip),
        '{{"reqSignature":{{}},"txn":{{"data":{{"data":{{"alias":"Node4","blskey":"2zN3bHM1m4rLz54MJHYSwvqzPchYp8jkHswveCLAEJVcX6Mm1wHQD1SkPYMzUDTZvWvhuE6VNAkK3KxVeEmsanSmvjVkReDeBEMxeDaayjcZjFGPydyey1qxBHmTvAnBKoPydvuTAqx5f7YNNRAdeLmUi99gERUU7TD8KfAa6MpQ9bw","blskey_pop":"RPLagxaR5xdimFzwmzYnz4ZhWtYQEj8iR5ZU53T2gitPCyCHQneUn2Huc4oeLd2B2HzkGnjAff4hWTJT6C7qHYB1Mv2wU5iHHGFWkhnTX9WsEAbunJCV2qcaXScKj4tTfvdDKfLiVuU2av6hbsMztirRze7LvYBkRHV3tGwyCptsrP","client_ip":"{}","client_port":9708,"node_ip":"{}","node_port":9707,"services":["VALIDATOR"]}},"dest":"4PS3EDQ3dW1tci1Bp6543CfuuebjFrg36kLAUcskGfaA"}},"metadata":{{"from":"TWwCRQRZ2ZHMJFn9TzLp7W"}},"type":"0"}},"txnMetadata":{{"seqNo":4,"txnId":"aa5e817d7cc626170eca175822029339a444eb0ee8f0bd20d3b0b76e566fb008"}},"ver":"1"}}'.format(
            pool_ip, pool_ip)
    ])

In [24]:
def save_pool_genesis_txn_file(path):
    data = pool_genesis_txn_data()

    path.parent.mkdir(parents=True, exist_ok=True)

    with open(str(path), "w+") as f:
        f.writelines(data)

In [25]:
async def ensure_previous_request_applied(pool_handle, checker_request, checker):
    for _ in range(3):
        response = json.loads(await ledger.submit_request(pool_handle, checker_request))
        try:
            if checker(response):
                return json.dumps(response)
        except TypeError:
            pass
        time.sleep(5)

# 3. 핵심함수 정의

In [26]:
def wallet_config(operation, wallet_config_str):
    return wallet_config_str

In [27]:
def wallet_credentials(operation, wallet_credentials_str):
    return wallet_credentials_str

In [28]:
async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(wallet_config("create", identity['wallet_config']),
                                   wallet_credentials("create", identity['wallet_credentials']))
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    identity['wallet'] = await wallet.open_wallet(wallet_config("open", identity['wallet_config']),
                                                  wallet_credentials("open", identity['wallet_credentials']))

In [29]:
async def getting_verinym(from_, to):
    await create_wallet(to)

    (to['did'], to['key']) = await did.create_and_store_my_did(to['wallet'], "{}")

    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }

    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])

In [30]:
async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)

In [31]:
async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)

In [32]:
async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)

In [33]:
async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    get_schema_response = await ensure_previous_request_applied(
        pool_handle, get_schema_request, lambda response: response['result']['data'] is not None)
    return await ledger.parse_get_schema_response(get_schema_response)

In [34]:
async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = \
        await ensure_previous_request_applied(pool_handle, get_cred_def_request,
                                              lambda response: response['result']['data'] is not None)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)

In [35]:
async def get_credential_for_referent(search_handle, referent):
    credentials = json.loads(
        await anoncreds.prover_fetch_credentials_for_proof_req(search_handle, referent, 10))
    return credentials[0]['cred_info']

In [36]:
def get_timestamp_for_attribute(cred_for_attribute, revoc_states):
    if cred_for_attribute['rev_reg_id'] in revoc_states:
        return int(next(iter(revoc_states[cred_for_attribute['rev_reg_id']])))
    else:
        return None

In [37]:
async def prover_get_entities_from_ledger(pool_handle, _did, identifiers, actor, timestamp_from=None,
                                          timestamp_to=None):
    schemas = {}
    cred_defs = {}
    rev_states = {}
    for item in identifiers.values():
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Claim Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_id' in item and item['rev_reg_id'] is not None:
            # Create Revocations States
            print("\"{}\" -> Get Revocation Registry Definition from Ledger".format(actor))
            get_revoc_reg_def_request = await ledger.build_get_revoc_reg_def_request(_did, item['rev_reg_id'])

            get_revoc_reg_def_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_def_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, revoc_reg_def_json) = await ledger.parse_get_revoc_reg_def_response(get_revoc_reg_def_response)

            print("\"{}\" -> Get Revocation Registry Delta from Ledger".format(actor))
            if not timestamp_to: timestamp_to = int(time.time())
            get_revoc_reg_delta_request = \
                await ledger.build_get_revoc_reg_delta_request(_did, item['rev_reg_id'], timestamp_from, timestamp_to)
            get_revoc_reg_delta_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_delta_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, revoc_reg_delta_json, t) = \
                await ledger.parse_get_revoc_reg_delta_response(get_revoc_reg_delta_response)

            tails_reader_config = json.dumps(
                {'base_dir': dirname(json.loads(revoc_reg_def_json)['value']['tailsLocation']),
                 'uri_pattern': ''})
            blob_storage_reader_cfg_handle = await blob_storage.open_reader('default', tails_reader_config)

            print('%s - Create Revocation State', actor)
            rev_state_json = \
                await anoncreds.create_revocation_state(blob_storage_reader_cfg_handle, revoc_reg_def_json,
                                                        revoc_reg_delta_json, t, item['cred_rev_id'])
            rev_states[rev_reg_id] = {t: json.loads(rev_state_json)}

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_states)

In [38]:
async def verifier_get_entities_from_ledger(pool_handle, _did, identifiers, actor, timestamp=None):
    schemas = {}
    cred_defs = {}
    rev_reg_defs = {}
    rev_regs = {}
    for item in identifiers:
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Claim Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_id' in item and item['rev_reg_id'] is not None:
            # Get Revocation Definitions and Revocation Registries
            print("\"{}\" -> Get Revocation Definition from Ledger".format(actor))
            get_revoc_reg_def_request = await ledger.build_get_revoc_reg_def_request(_did, item['rev_reg_id'])

            get_revoc_reg_def_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_def_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, revoc_reg_def_json) = await ledger.parse_get_revoc_reg_def_response(get_revoc_reg_def_response)

            print("\"{}\" -> Get Revocation Registry from Ledger".format(actor))
            if not timestamp: timestamp = item['timestamp']
            get_revoc_reg_request = \
                await ledger.build_get_revoc_reg_request(_did, item['rev_reg_id'], timestamp)
            get_revoc_reg_response = \
                await ensure_previous_request_applied(pool_handle, get_revoc_reg_request,
                                                      lambda response: response['result']['data'] is not None)
            (rev_reg_id, rev_reg_json, timestamp2) = await ledger.parse_get_revoc_reg_response(get_revoc_reg_response)

            rev_regs[rev_reg_id] = {timestamp2: json.loads(rev_reg_json)}
            rev_reg_defs[rev_reg_id] = json.loads(revoc_reg_def_json)

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_reg_defs), json.dumps(rev_regs)

# 4. 시나리오 시작

In [41]:
print("Getting started -> started")

pool_ = {
    'name': 'pool1'
}
print("Open Pool Ledger: {}".format(pool_['name']))
pool_['genesis_txn_path'] = get_pool_genesis_txn_path(pool_['name'])
pool_['config'] = json.dumps({"genesis_txn": str(pool_['genesis_txn_path'])})

# Set protocol version 2 to work with Indy Node 1.4
await pool.set_protocol_version(PROTOCOL_VERSION)

try:
    await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
except IndyError as ex:
    if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
        pass
pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

Getting started -> started
Open Pool Ledger: pool1


In [42]:
print("==============================")
print("=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==")
print("------------------------------")

steward = {
    'name': "Sovrin Steward",
    'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
    'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
    'pool': pool_['handle'],
    'seed': '000000000000000000000000Steward1'
}

await create_wallet(steward)

print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
steward['did_info'] = json.dumps({'seed': steward['seed']})
steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==
------------------------------
"Sovrin Steward" -> Create wallet
"Sovrin Steward" -> Create and store in Wallet DID from seed


In [43]:
print("==============================")
print("== Getting Trust Anchor credentials - Government getting Verinym  ==")
print("------------------------------")

government = {
    'name': 'Government',
    'wallet_config': json.dumps({'id': 'government_wallet'}),
    'wallet_credentials': json.dumps({'key': 'government_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, government)

== Getting Trust Anchor credentials - Government getting Verinym  ==
------------------------------
"Government" -> Create wallet


In [44]:
print("==============================")
print("== Getting Trust Anchor credentials - Faber getting Verinym  ==")
print("------------------------------")

faber = {
    'name': 'Faber',
    'wallet_config': json.dumps({'id': 'faber_wallet'}),
    'wallet_credentials': json.dumps({'key': 'faber_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, faber)

== Getting Trust Anchor credentials - Faber getting Verinym  ==
------------------------------
"Faber" -> Create wallet


In [45]:
print("==============================")
print("== Getting Trust Anchor credentials - Acme getting Verinym  ==")
print("------------------------------")

acme = {
    'name': 'Acme',
    'wallet_config': json.dumps({'id': 'acme_wallet'}),
    'wallet_credentials': json.dumps({'key': 'acme_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, acme)

== Getting Trust Anchor credentials - Acme getting Verinym  ==
------------------------------
"Acme" -> Create wallet


In [46]:
print("==============================")
print("== Getting Trust Anchor credentials - Thrift getting Verinym  ==")
print("------------------------------")

thrift = {
    'name': 'Thrift',
    'wallet_config': json.dumps({'id': 'thrift_wallet'}),
    'wallet_credentials': json.dumps({'key': 'thrift_wallet_key'}),
    'pool': pool_['handle'],
    'role': 'TRUST_ANCHOR'
}

await getting_verinym(steward, thrift)

== Getting Trust Anchor credentials - Thrift getting Verinym  ==
------------------------------
"Thrift" -> Create wallet


In [47]:
print("==============================")
print("=== Credential Schemas Setup ==")
print("------------------------------")

print("\"Government\" -> Create \"Job-Certificate\" Schema")
job_certificate = {
    'name': 'Job-Certificate',
    'version': '0.2',
    'attributes': ['first_name', 'last_name', 'salary', 'employee_status', 'experience']
}
(government['job_certificate_schema_id'], government['job_certificate_schema']) = \
    await anoncreds.issuer_create_schema(government['did'], job_certificate['name'], job_certificate['version'],
                                         json.dumps(job_certificate['attributes']))
job_certificate_schema_id = government['job_certificate_schema_id']

print("\"Government\" -> Send \"Job-Certificate\" Schema to Ledger")
await send_schema(government['pool'], government['wallet'], government['did'], government['job_certificate_schema'])

print("\"Government\" -> Create \"Transcript\" Schema")
transcript = {
    'name': 'Transcript',
    'version': '1.2',
    'attributes': ['first_name', 'last_name', 'degree', 'status', 'year', 'average', 'ssn']
}
(government['transcript_schema_id'], government['transcript_schema']) = \
    await anoncreds.issuer_create_schema(government['did'], transcript['name'], transcript['version'],
                                         json.dumps(transcript['attributes']))
transcript_schema_id = government['transcript_schema_id']

print("\"Government\" -> Send \"Transcript\" Schema to Ledger")
await send_schema(government['pool'], government['wallet'], government['did'], government['transcript_schema'])

time.sleep(1)  # sleep 1 second before getting schema

=== Credential Schemas Setup ==
------------------------------
"Government" -> Create "Job-Certificate" Schema
"Government" -> Send "Job-Certificate" Schema to Ledger
"Government" -> Create "Transcript" Schema
"Government" -> Send "Transcript" Schema to Ledger


In [48]:
print("==============================")
print("=== Faber Credential Definition Setup ==")
print("------------------------------")

print("\"Faber\" -> Get \"Transcript\" Schema from Ledger")
(faber['transcript_schema_id'], faber['transcript_schema']) = \
    await get_schema(faber['pool'], faber['did'], transcript_schema_id)

print("\"Faber\" -> Create and store in Wallet \"Faber Transcript\" Credential Definition")
transcript_cred_def = {
    'tag': 'TAG1',
    'type': 'CL',
    'config': {"support_revocation": False}
}
(faber['transcript_cred_def_id'], faber['transcript_cred_def']) = \
    await anoncreds.issuer_create_and_store_credential_def(faber['wallet'], faber['did'],
                                                           faber['transcript_schema'], transcript_cred_def['tag'],
                                                           transcript_cred_def['type'],
                                                           json.dumps(transcript_cred_def['config']))

print("\"Faber\" -> Send  \"Faber Transcript\" Credential Definition to Ledger")
await send_cred_def(faber['pool'], faber['wallet'], faber['did'], faber['transcript_cred_def'])

=== Faber Credential Definition Setup ==
------------------------------
"Faber" -> Get "Transcript" Schema from Ledger
"Faber" -> Create and store in Wallet "Faber Transcript" Credential Definition
"Faber" -> Send  "Faber Transcript" Credential Definition to Ledger


In [49]:
print("==============================")
print("=== Acme Credential Definition Setup ==")
print("------------------------------")

print("\"Acme\" -> Get from Ledger \"Job-Certificate\" Schema")
(acme['job_certificate_schema_id'], acme['job_certificate_schema']) = \
    await get_schema(acme['pool'], acme['did'], job_certificate_schema_id)

print("\"Acme\" -> Create and store in Wallet \"Acme Job-Certificate\" Credential Definition")
job_certificate_cred_def = {
    'tag': 'TAG1',
    'type': 'CL',
    'config': {"support_revocation": True}
}
(acme['job_certificate_cred_def_id'], acme['job_certificate_cred_def']) = \
    await anoncreds.issuer_create_and_store_credential_def(acme['wallet'], acme['did'],
                                                           acme['job_certificate_schema'],
                                                           job_certificate_cred_def['tag'],
                                                           job_certificate_cred_def['type'],
                                                           json.dumps(job_certificate_cred_def['config']))

print("\"Acme\" -> Send \"Acme Job-Certificate\" Credential Definition to Ledger")
await send_cred_def(acme['pool'], acme['wallet'], acme['did'], acme['job_certificate_cred_def'])

print("\"Acme\" -> Creates Revocation Registry")
acme['tails_writer_config'] = json.dumps({'base_dir': "/tmp/indy_acme_tails", 'uri_pattern': ''})
tails_writer = await blob_storage.open_writer('default', acme['tails_writer_config'])
(acme['revoc_reg_id'], acme['revoc_reg_def'], acme['revoc_reg_entry']) = \
    await anoncreds.issuer_create_and_store_revoc_reg(acme['wallet'], acme['did'], 'CL_ACCUM', 'TAG1',
                                                      acme['job_certificate_cred_def_id'],
                                                      json.dumps({'max_cred_num': 5,
                                                                  'issuance_type': 'ISSUANCE_ON_DEMAND'}),
                                                      tails_writer)

print("\"Acme\" -> Post Revocation Registry Definition to Ledger")
acme['revoc_reg_def_request'] = await ledger.build_revoc_reg_def_request(acme['did'], acme['revoc_reg_def'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_def_request'])

print("\"Acme\" -> Post Revocation Registry Entry to Ledger")
acme['revoc_reg_entry_request'] = \
    await ledger.build_revoc_reg_entry_request(acme['did'], acme['revoc_reg_id'], 'CL_ACCUM',
                                               acme['revoc_reg_entry'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_entry_request'])

=== Acme Credential Definition Setup ==
------------------------------
"Acme" -> Get from Ledger "Job-Certificate" Schema
"Acme" -> Create and store in Wallet "Acme Job-Certificate" Credential Definition
"Acme" -> Send "Acme Job-Certificate" Credential Definition to Ledger
"Acme" -> Creates Revocation Registry
"Acme" -> Post Revocation Registry Definition to Ledger
"Acme" -> Post Revocation Registry Entry to Ledger


'{"op":"REPLY","result":{"txnMetadata":{"txnTime":1627594708,"seqNo":121,"txnId":"5:QbgJQUj1UVrFV28xQNn1Gs:4:QbgJQUj1UVrFV28xQNn1Gs:3:CL:116:TAG1:CL_ACCUM:TAG1"},"auditPath":["AfEQoRJ9A4AgcYPUhtmSUp74feBFrVA9SufFkdkMhTah","XPtMEgrxvSdzrZRiTzM5nWSVVfAc6LkQqMVrcuwcKwN","6dvhke25RyaL7KEDjo6Qz8d8gmwRbmJ3akurLQJrWYyr","ChXSt8Tk59pd6re6jEL1LiDxHRrzQ2BFnpms9dMCANFW"],"reqSignature":{"type":"ED25519","values":[{"value":"4crEvFabRHbcRR7X4sDgPtdWubsfLaZ9PFT3Z4GAAgGxf55GsfEGSxSaFLJhdLU7vixVPR4SuJC9yQj9GdAnKbC2","from":"QbgJQUj1UVrFV28xQNn1Gs"}]},"rootHash":"AV7ctbomPYcpGN7ULiQm7MSspqE4nqAvDoSQwk981tmL","ver":"1","txn":{"type":"114","data":{"revocDefType":"CL_ACCUM","revocRegDefId":"QbgJQUj1UVrFV28xQNn1Gs:4:QbgJQUj1UVrFV28xQNn1Gs:3:CL:116:TAG1:CL_ACCUM:TAG1","value":{"accum":"1 0000000000000000000000000000000000000000000000000000000000000000 1 0000000000000000000000000000000000000000000000000000000000000000 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000

In [50]:
print("==============================")
print("=== Getting Transcript with Faber ==")
print("==============================")
print("== Alice setup ==")
print("------------------------------")

alice = {
    'name': 'Alice',
    'wallet_config': json.dumps({'id': 'alice_wallet'}),
    'wallet_credentials': json.dumps({'key': 'alice_wallet_key'}),
    'pool': pool_['handle'],
}
await create_wallet(alice)
(alice['did'], alice['key']) = await did.create_and_store_my_did(alice['wallet'], "{}")

=== Getting Transcript with Faber ==
== Alice setup ==
------------------------------
"Alice" -> Create wallet


In [51]:
print("==============================")
print("== Getting Transcript with Faber - Getting Transcript Credential ==")
print("------------------------------")

print("\"Faber\" -> Create \"Transcript\" Credential Offer for Alice")
faber['transcript_cred_offer'] = \
    await anoncreds.issuer_create_credential_offer(faber['wallet'], faber['transcript_cred_def_id'])

print("\"Faber\" -> Send \"Transcript\" Credential Offer to Alice")
alice['transcript_cred_offer'] = faber['transcript_cred_offer']
transcript_cred_offer_object = json.loads(alice['transcript_cred_offer'])

alice['transcript_schema_id'] = transcript_cred_offer_object['schema_id']
alice['transcript_cred_def_id'] = transcript_cred_offer_object['cred_def_id']

print("\"Alice\" -> Create and store \"Alice\" Master Secret in Wallet")
alice['master_secret_id'] = await anoncreds.prover_create_master_secret(alice['wallet'], None)

print("\"Alice\" -> Get \"Faber Transcript\" Credential Definition from Ledger")
(alice['faber_transcript_cred_def_id'], alice['faber_transcript_cred_def']) = \
    await get_cred_def(alice['pool'], alice['did'], alice['transcript_cred_def_id'])

print("\"Alice\" -> Create \"Transcript\" Credential Request for Faber")
(alice['transcript_cred_request'], alice['transcript_cred_request_metadata']) = \
    await anoncreds.prover_create_credential_req(alice['wallet'], alice['did'],
                                                 alice['transcript_cred_offer'], alice['faber_transcript_cred_def'],
                                                 alice['master_secret_id'])

print("\"Alice\" -> Send \"Transcript\" Credential Request to Faber")
faber['transcript_cred_request'] = alice['transcript_cred_request']

print("\"Faber\" -> Create \"Transcript\" Credential for Alice")
faber['alice_transcript_cred_values'] = json.dumps({
    "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
    "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
    "degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
    "status": {"raw": "graduated", "encoded": "2213454313412354"},
    "ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
    "year": {"raw": "2015", "encoded": "2015"},
    "average": {"raw": "5", "encoded": "5"}
})
faber['transcript_cred'], _, _ = \
    await anoncreds.issuer_create_credential(faber['wallet'], faber['transcript_cred_offer'],
                                             faber['transcript_cred_request'],
                                             faber['alice_transcript_cred_values'], None, None)

print("\"Faber\" -> Send \"Transcript\" Credential to Alice")
alice['transcript_cred'] = faber['transcript_cred']

print("\"Alice\" -> Store \"Transcript\" Credential from Faber")
_, alice['transcript_cred_def'] = await get_cred_def(alice['pool'], alice['did'],
                                                     alice['transcript_cred_def_id'])

await anoncreds.prover_store_credential(alice['wallet'], None, alice['transcript_cred_request_metadata'],
                                        alice['transcript_cred'], alice['transcript_cred_def'], None)

== Getting Transcript with Faber - Getting Transcript Credential ==
------------------------------
"Faber" -> Create "Transcript" Credential Offer for Alice
"Faber" -> Send "Transcript" Credential Offer to Alice
"Alice" -> Create and store "Alice" Master Secret in Wallet
"Alice" -> Get "Faber Transcript" Credential Definition from Ledger
"Alice" -> Create "Transcript" Credential Request for Faber
"Alice" -> Send "Transcript" Credential Request to Faber
"Faber" -> Create "Transcript" Credential for Alice
"Faber" -> Send "Transcript" Credential to Alice
"Alice" -> Store "Transcript" Credential from Faber


'c8635eac-433e-4d90-bcb8-33c6473ec43b'

In [52]:
print("==============================")
print("== Apply for the job with Acme - Transcript proving ==")
print("------------------------------")

print("\"Acme\" -> Create \"Job-Application\" Proof Request")
nonce = await anoncreds.generate_nonce()
acme['job_application_proof_request'] = json.dumps({
    'nonce': nonce,
    'name': 'Job-Application',
    'version': '0.1',
    'requested_attributes': {
        'attr1_referent': {
            'name': 'first_name'
        },
        'attr2_referent': {
            'name': 'last_name'
        },
        'attr3_referent': {
            'name': 'degree',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr4_referent': {
            'name': 'status',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr5_referent': {
            'name': 'ssn',
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        },
        'attr6_referent': {
            'name': 'phone_number'
        }
    },
    'requested_predicates': {
        'predicate1_referent': {
            'name': 'average',
            'p_type': '>=',
            'p_value': 4,
            'restrictions': [{'cred_def_id': faber['transcript_cred_def_id']}]
        }
    }
})

print("\"Acme\" -> Send \"Job-Application\" Proof Request to Alice")
alice['job_application_proof_request'] = acme['job_application_proof_request']

print("\"Alice\" -> Get credentials for \"Job-Application\" Proof Request")

search_for_job_application_proof_request = \
    await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                            alice['job_application_proof_request'], None)

cred_for_attr1 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr1_referent')
cred_for_attr2 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr2_referent')
cred_for_attr3 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr3_referent')
cred_for_attr4 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr4_referent')
cred_for_attr5 = await get_credential_for_referent(search_for_job_application_proof_request, 'attr5_referent')
cred_for_predicate1 = \
    await get_credential_for_referent(search_for_job_application_proof_request, 'predicate1_referent')

await anoncreds.prover_close_credentials_search_for_proof_req(search_for_job_application_proof_request)

alice['creds_for_job_application_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                            cred_for_attr2['referent']: cred_for_attr2,
                                            cred_for_attr3['referent']: cred_for_attr3,
                                            cred_for_attr4['referent']: cred_for_attr4,
                                            cred_for_attr5['referent']: cred_for_attr5,
                                            cred_for_predicate1['referent']: cred_for_predicate1}

alice['schemas_for_job_application'], alice['cred_defs_for_job_application'], \
alice['revoc_states_for_job_application'] = \
    await prover_get_entities_from_ledger(alice['pool'], alice['did'],
                                          alice['creds_for_job_application_proof'], alice['name'])

print("\"Alice\" -> Create \"Job-Application\" Proof")
alice['job_application_requested_creds'] = json.dumps({
    'self_attested_attributes': {
        'attr1_referent': 'Alice',
        'attr2_referent': 'Garcia',
        'attr6_referent': '123-45-6789'
    },
    'requested_attributes': {
        'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True},
        'attr4_referent': {'cred_id': cred_for_attr4['referent'], 'revealed': True},
        'attr5_referent': {'cred_id': cred_for_attr5['referent'], 'revealed': True},
    },
    'requested_predicates': {'predicate1_referent': {'cred_id': cred_for_predicate1['referent']}}
})

alice['job_application_proof'] = \
    await anoncreds.prover_create_proof(alice['wallet'], alice['job_application_proof_request'],
                                        alice['job_application_requested_creds'], alice['master_secret_id'],
                                        alice['schemas_for_job_application'],
                                        alice['cred_defs_for_job_application'],
                                        alice['revoc_states_for_job_application'])

print("\"Alice\" -> Send \"Job-Application\" Proof to Acme")
acme['job_application_proof'] = alice['job_application_proof']

job_application_proof_object = json.loads(acme['job_application_proof'])

acme['schemas_for_job_application'], acme['cred_defs_for_job_application'], \
acme['revoc_ref_defs_for_job_application'], acme['revoc_regs_for_job_application'] = \
    await verifier_get_entities_from_ledger(acme['pool'], acme['did'],
                                            job_application_proof_object['identifiers'], acme['name'])

print("\"Acme\" -> Verify \"Job-Application\" Proof from Alice")
assert 'Bachelor of Science, Marketing' == \
       job_application_proof_object['requested_proof']['revealed_attrs']['attr3_referent']['raw']
assert 'graduated' == \
       job_application_proof_object['requested_proof']['revealed_attrs']['attr4_referent']['raw']
assert '123-45-6789' == \
       job_application_proof_object['requested_proof']['revealed_attrs']['attr5_referent']['raw']

assert 'Alice' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr1_referent']
assert 'Garcia' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr2_referent']
assert '123-45-6789' == job_application_proof_object['requested_proof']['self_attested_attrs']['attr6_referent']

assert await anoncreds.verifier_verify_proof(acme['job_application_proof_request'], acme['job_application_proof'],
                                             acme['schemas_for_job_application'],
                                             acme['cred_defs_for_job_application'],
                                             acme['revoc_ref_defs_for_job_application'],
                                             acme['revoc_regs_for_job_application'])

== Apply for the job with Acme - Transcript proving ==
------------------------------
"Acme" -> Create "Job-Application" Proof Request
"Acme" -> Send "Job-Application" Proof Request to Alice
"Alice" -> Get credentials for "Job-Application" Proof Request
"Alice" -> Get Schema from Ledger
"Alice" -> Get Claim Definition from Ledger
"Alice" -> Create "Job-Application" Proof
"Alice" -> Send "Job-Application" Proof to Acme
"Acme" -> Get Schema from Ledger
"Acme" -> Get Claim Definition from Ledger
"Acme" -> Verify "Job-Application" Proof from Alice


In [53]:
print("==============================")
print("== Apply for the job with Acme - Getting Job-Certificate Credential ==")
print("------------------------------")

print("\"Acme\" -> Create \"Job-Certificate\" Credential Offer for Alice")
acme['job_certificate_cred_offer'] = \
    await anoncreds.issuer_create_credential_offer(acme['wallet'], acme['job_certificate_cred_def_id'])

print("\"Acme\" -> Send \"Job-Certificate\" Credential Offer to Alice")
alice['job_certificate_cred_offer'] = acme['job_certificate_cred_offer']

job_certificate_cred_offer_object = json.loads(alice['job_certificate_cred_offer'])

print("\"Alice\" -> Get \"Acme Job-Certificate\" Credential Definition from Ledger")
(alice['acme_job_certificate_cred_def_id'], alice['acme_job_certificate_cred_def']) = \
    await get_cred_def(alice['pool'], alice['did'], job_certificate_cred_offer_object['cred_def_id'])

print("\"Alice\" -> Create and store in Wallet \"Job-Certificate\" Credential Request for Acme")
(alice['job_certificate_cred_request'], alice['job_certificate_cred_request_metadata']) = \
    await anoncreds.prover_create_credential_req(alice['wallet'], alice['did'],
                                                 alice['job_certificate_cred_offer'],
                                                 alice['acme_job_certificate_cred_def'], alice['master_secret_id'])

print("\"Alice\" -> Send \"Job-Certificate\" Credential Request to Acme")
alice['job_certificate_cred_values'] = json.dumps({
    "first_name": {"raw": "Alice", "encoded": "245712572474217942457235975012103335"},
    "last_name": {"raw": "Garcia", "encoded": "312643218496194691632153761283356127"},
    "employee_status": {"raw": "Permanent", "encoded": "2143135425425143112321314321"},
    "salary": {"raw": "2400", "encoded": "2400"},
    "experience": {"raw": "10", "encoded": "10"}
})
acme['job_certificate_cred_request'] = alice['job_certificate_cred_request']
acme['job_certificate_cred_values'] = alice['job_certificate_cred_values']

print("\"Acme\" -> Create \"Job-Certificate\" Credential for Alice")
acme['blob_storage_reader_cfg_handle'] = await blob_storage.open_reader('default', acme['tails_writer_config'])
acme['job_certificate_cred'], acme['job_certificate_cred_rev_id'], acme['alice_cert_rev_reg_delta'] = \
    await anoncreds.issuer_create_credential(acme['wallet'], acme['job_certificate_cred_offer'],
                                             acme['job_certificate_cred_request'],
                                             acme['job_certificate_cred_values'],
                                             acme['revoc_reg_id'],
                                             acme['blob_storage_reader_cfg_handle'])

print("\"Acme\" -> Post Revocation Registry Delta to Ledger")
acme['revoc_reg_entry_req'] = \
    await ledger.build_revoc_reg_entry_request(acme['did'], acme['revoc_reg_id'], 'CL_ACCUM',
                                               acme['alice_cert_rev_reg_delta'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_entry_req'])

print("\"Acme\" -> Send \"Job-Certificate\" Credential to Alice")
alice['job_certificate_cred'] = acme['job_certificate_cred']
job_certificate_cred_object = json.loads(alice['job_certificate_cred'])

print("\"Alice\" -> Gets RevocationRegistryDefinition for \"Job-Certificate\" Credential from Acme")
alice['acme_revoc_reg_des_req'] = \
    await ledger.build_get_revoc_reg_def_request(alice['did'],
                                                 job_certificate_cred_object['rev_reg_id'])
alice['acme_revoc_reg_des_resp'] = \
    await ensure_previous_request_applied(alice['pool'], alice['acme_revoc_reg_des_req'],
                                          lambda response: response['result']['data'] is not None)
(alice['acme_revoc_reg_def_id'], alice['acme_revoc_reg_def_json']) = \
    await ledger.parse_get_revoc_reg_def_response(alice['acme_revoc_reg_des_resp'])

print("\"Alice\" -> Store \"Job-Certificate\" Credential")
await anoncreds.prover_store_credential(alice['wallet'], None, alice['job_certificate_cred_request_metadata'],
                                        alice['job_certificate_cred'],
                                        alice['acme_job_certificate_cred_def'], alice['acme_revoc_reg_def_json'])

== Apply for the job with Acme - Getting Job-Certificate Credential ==
------------------------------
"Acme" -> Create "Job-Certificate" Credential Offer for Alice
"Acme" -> Send "Job-Certificate" Credential Offer to Alice
"Alice" -> Get "Acme Job-Certificate" Credential Definition from Ledger
"Alice" -> Create and store in Wallet "Job-Certificate" Credential Request for Acme
"Alice" -> Send "Job-Certificate" Credential Request to Acme
"Acme" -> Create "Job-Certificate" Credential for Alice
"Acme" -> Post Revocation Registry Delta to Ledger
"Acme" -> Send "Job-Certificate" Credential to Alice
"Alice" -> Gets RevocationRegistryDefinition for "Job-Certificate" Credential from Acme
"Alice" -> Store "Job-Certificate" Credential


'5b5c1424-a9f4-4e72-b10e-8c58a20064a0'

In [54]:
print("==============================")
print("=== Apply for the loan with Thrift ==")
print("==============================")


async def apply_loan_basic():
    # This method will be called twice: once with a valid Job-Certificate and
    # the second time after the Job-Certificate has been revoked.
    print("==============================")
    print("== Apply for the loan with Thrift - Job-Certificate proving  ==")
    print("------------------------------")

    print("\"Thrift\" -> Create \"Loan-Application-Basic\" Proof Request")
    nonce = await anoncreds.generate_nonce()
    thrift['apply_loan_proof_request'] = json.dumps({
        'nonce': nonce,
        'name': 'Loan-Application-Basic',
        'version': '0.1',
        'requested_attributes': {
            'attr1_referent': {
                'name': 'employee_status',
                'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
            }
        },
        'requested_predicates': {
            'predicate1_referent': {
                'name': 'salary',
                'p_type': '>=',
                'p_value': 2000,
                'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
            },
            'predicate2_referent': {
                'name': 'experience',
                'p_type': '>=',
                'p_value': 1,
                'restrictions': [{'cred_def_id': acme['job_certificate_cred_def_id']}]
            }
        },
        'non_revoked': {'to': int(time.time())}
    })

    print("\"Thrift\" -> Send \"Loan-Application-Basic\" Proof Request to Alice")
    alice['apply_loan_proof_request'] = thrift['apply_loan_proof_request']

    print("\"Alice\" -> Get credentials for \"Loan-Application-Basic\" Proof Request")

    search_for_apply_loan_proof_request = \
        await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                                alice['apply_loan_proof_request'], None)

    cred_for_attr1 = await get_credential_for_referent(search_for_apply_loan_proof_request, 'attr1_referent')
    cred_for_predicate1 = await get_credential_for_referent(search_for_apply_loan_proof_request,
                                                            'predicate1_referent')
    cred_for_predicate2 = await get_credential_for_referent(search_for_apply_loan_proof_request,
                                                            'predicate2_referent')

    await anoncreds.prover_close_credentials_search_for_proof_req(search_for_apply_loan_proof_request)

    alice['creds_for_apply_loan_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                           cred_for_predicate1['referent']: cred_for_predicate1,
                                           cred_for_predicate2['referent']: cred_for_predicate2}

    requested_timestamp = int(json.loads(thrift['apply_loan_proof_request'])['non_revoked']['to'])
    alice['schemas_for_loan_app'], alice['cred_defs_for_loan_app'], alice['revoc_states_for_loan_app'] = \
        await prover_get_entities_from_ledger(alice['pool'], alice['did'],
                                              alice['creds_for_apply_loan_proof'],
                                              alice['name'], None, requested_timestamp)

    print("\"Alice\" -> Create \"Loan-Application-Basic\" Proof")
    revoc_states_for_loan_app = json.loads(alice['revoc_states_for_loan_app'])
    timestamp_for_attr1 = get_timestamp_for_attribute(cred_for_attr1, revoc_states_for_loan_app)
    timestamp_for_predicate1 = get_timestamp_for_attribute(cred_for_predicate1, revoc_states_for_loan_app)
    timestamp_for_predicate2 = get_timestamp_for_attribute(cred_for_predicate2, revoc_states_for_loan_app)
    alice['apply_loan_requested_creds'] = json.dumps({
        'self_attested_attributes': {},
        'requested_attributes': {
            'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True,
                               'timestamp': timestamp_for_attr1}
        },
        'requested_predicates': {
            'predicate1_referent': {'cred_id': cred_for_predicate1['referent'],
                                    'timestamp': timestamp_for_predicate1},
            'predicate2_referent': {'cred_id': cred_for_predicate2['referent'],
                                    'timestamp': timestamp_for_predicate2}
        }
    })
    alice['apply_loan_proof'] = \
        await anoncreds.prover_create_proof(alice['wallet'], alice['apply_loan_proof_request'],
                                            alice['apply_loan_requested_creds'], alice['master_secret_id'],
                                            alice['schemas_for_loan_app'], alice['cred_defs_for_loan_app'],
                                            alice['revoc_states_for_loan_app'])

    print("\"Alice\" -> Send \"Loan-Application-Basic\" Proof to Thrift")
    thrift['apply_loan_proof'] = alice['apply_loan_proof']
    apply_loan_proof_object = json.loads(thrift['apply_loan_proof'])

    print("\"Thrift\" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger"
                " required for Proof verifying")

    thrift['schemas_for_loan_app'], thrift['cred_defs_for_loan_app'], thrift['revoc_defs_for_loan_app'], \
    thrift['revoc_regs_for_loan_app'] = \
        await verifier_get_entities_from_ledger(thrift['pool'], thrift['did'],
                                                apply_loan_proof_object['identifiers'],
                                                thrift['name'], requested_timestamp)

    print("\"Thrift\" -> Verify \"Loan-Application-Basic\" Proof from Alice")
    assert 'Permanent' == \
           apply_loan_proof_object['requested_proof']['revealed_attrs']['attr1_referent']['raw']


await apply_loan_basic()

assert await anoncreds.verifier_verify_proof(thrift['apply_loan_proof_request'],
                                             thrift['apply_loan_proof'],
                                             thrift['schemas_for_loan_app'],
                                             thrift['cred_defs_for_loan_app'],
                                             thrift['revoc_defs_for_loan_app'],
                                             thrift['revoc_regs_for_loan_app'])

=== Apply for the loan with Thrift ==
== Apply for the loan with Thrift - Job-Certificate proving  ==
------------------------------
"Thrift" -> Create "Loan-Application-Basic" Proof Request
"Thrift" -> Send "Loan-Application-Basic" Proof Request to Alice
"Alice" -> Get credentials for "Loan-Application-Basic" Proof Request
"Alice" -> Get Schema from Ledger
"Alice" -> Get Claim Definition from Ledger
"Alice" -> Get Revocation Registry Definition from Ledger
"Alice" -> Get Revocation Registry Delta from Ledger
%s - Create Revocation State Alice
"Alice" -> Create "Loan-Application-Basic" Proof
"Alice" -> Send "Loan-Application-Basic" Proof to Thrift
"Thrift" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger required for Proof verifying
"Thrift" -> Get Schema from Ledger
"Thrift" -> Get Claim Definition from Ledger
"Thrift" -> Get Revocation Definition from Ledger
"Thrift" -> Get Revocation Registry from Ledger
"Thrift" -> Verify "Loan-Application-Basic" Proof f

In [55]:
print("==============================")
print("== Apply for the loan with Thrift - Transcript and Job-Certificate proving  ==")
print("------------------------------")

print("\"Thrift\" -> Create \"Loan-Application-KYC\" Proof Request")
nonce = await anoncreds.generate_nonce()
thrift['apply_loan_kyc_proof_request'] = json.dumps({
    'nonce': nonce,
    'name': 'Loan-Application-KYC',
    'version': '0.1',
    'requested_attributes': {
        'attr1_referent': {'name': 'first_name'},
        'attr2_referent': {'name': 'last_name'},
        'attr3_referent': {'name': 'ssn'}
    },
    'requested_predicates': {}
})

print("\"Thrift\" -> Send \"Loan-Application-KYC\" Proof Request to Alice")
alice['apply_loan_kyc_proof_request'] = thrift['apply_loan_kyc_proof_request']

print("\"Alice\" -> Get credentials for \"Loan-Application-KYC\" Proof Request")

search_for_apply_loan_kyc_proof_request = \
    await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                            alice['apply_loan_kyc_proof_request'], None)

cred_for_attr1 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr1_referent')
cred_for_attr2 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr2_referent')
cred_for_attr3 = await get_credential_for_referent(search_for_apply_loan_kyc_proof_request, 'attr3_referent')

await anoncreds.prover_close_credentials_search_for_proof_req(search_for_apply_loan_kyc_proof_request)

alice['creds_for_apply_loan_kyc_proof'] = {cred_for_attr1['referent']: cred_for_attr1,
                                           cred_for_attr2['referent']: cred_for_attr2,
                                           cred_for_attr3['referent']: cred_for_attr3}

alice['schemas_for_loan_kyc_app'], alice['cred_defs_for_loan_kyc_app'], alice['revoc_states_for_loan_kyc_app'] = \
    await prover_get_entities_from_ledger(alice['pool'], alice['did'],
                                          alice['creds_for_apply_loan_kyc_proof'], alice['name'], )

print("\"Alice\" -> Create \"Loan-Application-KYC\" Proof")
revoc_states_for_loan_app = json.loads(alice['revoc_states_for_loan_kyc_app'])
timestamp_for_attr1 = get_timestamp_for_attribute(cred_for_attr1, revoc_states_for_loan_app)
timestamp_for_attr2 = get_timestamp_for_attribute(cred_for_attr2, revoc_states_for_loan_app)
timestamp_for_attr3 = get_timestamp_for_attribute(cred_for_attr3, revoc_states_for_loan_app)
alice['apply_loan_kyc_requested_creds'] = json.dumps({
    'self_attested_attributes': {},
    'requested_attributes': {
        'attr1_referent': {'cred_id': cred_for_attr1['referent'], 'revealed': True,
                           'timestamp': timestamp_for_attr1},
        'attr2_referent': {'cred_id': cred_for_attr2['referent'], 'revealed': True,
                           'timestamp': timestamp_for_attr2},
        'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True,
                           'timestamp': timestamp_for_attr3}
    },
    'requested_predicates': {}
})

alice['apply_loan_kyc_proof'] = \
    await anoncreds.prover_create_proof(alice['wallet'], alice['apply_loan_kyc_proof_request'],
                                        alice['apply_loan_kyc_requested_creds'], alice['master_secret_id'],
                                        alice['schemas_for_loan_kyc_app'], alice['cred_defs_for_loan_kyc_app'],
                                        alice['revoc_states_for_loan_kyc_app'])

print("\"Alice\" -> Send \"Loan-Application-KYC\" Proof to Thrift")
thrift['apply_loan_kyc_proof'] = alice['apply_loan_kyc_proof']
apply_loan_kyc_proof_object = json.loads(thrift['apply_loan_kyc_proof'])

print("\"Thrift\" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger"
            " required for Proof verifying")

thrift['schemas_for_loan_kyc_app'], thrift['cred_defs_for_loan_kyc_app'], thrift['revoc_defs_for_loan_kyc_app'], \
thrift['revoc_regs_for_loan_kyc_app'] = \
    await verifier_get_entities_from_ledger(thrift['pool'], thrift['did'],
                                            apply_loan_kyc_proof_object['identifiers'], thrift['name'])

print("\"Thrift\" -> Verify \"Loan-Application-KYC\" Proof from Alice")
assert 'Alice' == \
       apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr1_referent']['raw']
assert 'Garcia' == \
       apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr2_referent']['raw']
assert '123-45-6789' == \
       apply_loan_kyc_proof_object['requested_proof']['revealed_attrs']['attr3_referent']['raw']

assert await anoncreds.verifier_verify_proof(thrift['apply_loan_kyc_proof_request'],
                                             thrift['apply_loan_kyc_proof'],
                                             thrift['schemas_for_loan_kyc_app'],
                                             thrift['cred_defs_for_loan_kyc_app'],
                                             thrift['revoc_defs_for_loan_kyc_app'],
                                             thrift['revoc_regs_for_loan_kyc_app'])

print("==============================")

== Apply for the loan with Thrift - Transcript and Job-Certificate proving  ==
------------------------------
"Thrift" -> Create "Loan-Application-KYC" Proof Request
"Thrift" -> Send "Loan-Application-KYC" Proof Request to Alice
"Alice" -> Get credentials for "Loan-Application-KYC" Proof Request
"Alice" -> Get Schema from Ledger
"Alice" -> Get Claim Definition from Ledger
"Alice" -> Get Revocation Registry Definition from Ledger
"Alice" -> Get Revocation Registry Delta from Ledger
%s - Create Revocation State Alice
"Alice" -> Get Schema from Ledger
"Alice" -> Get Claim Definition from Ledger
"Alice" -> Create "Loan-Application-KYC" Proof
"Alice" -> Send "Loan-Application-KYC" Proof to Thrift
"Thrift" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger required for Proof verifying
"Thrift" -> Get Schema from Ledger
"Thrift" -> Get Claim Definition from Ledger
"Thrift" -> Get Revocation Definition from Ledger
"Thrift" -> Get Revocation Registry from Ledger
"Thrif

In [56]:
print("==============================")
print("== Credential revocation - Acme revokes Alice's Job-Certificate  ==")
print("------------------------------")

print("\"Acme\" - Revoke  credential")
acme['alice_cert_rev_reg_delta'] = \
    await anoncreds.issuer_revoke_credential(acme['wallet'],
                                             acme['blob_storage_reader_cfg_handle'],
                                             acme['revoc_reg_id'],
                                             acme['job_certificate_cred_rev_id'])

print("\"Acme\" - Post RevocationRegistryDelta to Ledger")
acme['revoc_reg_entry_req'] = \
    await ledger.build_revoc_reg_entry_request(acme['did'], acme['revoc_reg_id'], 'CL_ACCUM',
                                               acme['alice_cert_rev_reg_delta'])
await ledger.sign_and_submit_request(acme['pool'], acme['wallet'], acme['did'], acme['revoc_reg_entry_req'])

print("==============================")

== Credential revocation - Acme revokes Alice's Job-Certificate  ==
------------------------------
"Acme" - Revoke  credential
"Acme" - Post RevocationRegistryDelta to Ledger


In [57]:
print("==============================")
print("== Apply for the loan with Thrift again - Job-Certificate proving  ==")
print("------------------------------")

await apply_loan_basic()

assert not await anoncreds.verifier_verify_proof(thrift['apply_loan_proof_request'],
                                                 thrift['apply_loan_proof'],
                                                 thrift['schemas_for_loan_app'],
                                                 thrift['cred_defs_for_loan_app'],
                                                 thrift['revoc_defs_for_loan_app'],
                                                 thrift['revoc_regs_for_loan_app'])

print("==============================")

== Apply for the loan with Thrift again - Job-Certificate proving  ==
------------------------------
== Apply for the loan with Thrift - Job-Certificate proving  ==
------------------------------
"Thrift" -> Create "Loan-Application-Basic" Proof Request
"Thrift" -> Send "Loan-Application-Basic" Proof Request to Alice
"Alice" -> Get credentials for "Loan-Application-Basic" Proof Request
"Alice" -> Get Schema from Ledger
"Alice" -> Get Claim Definition from Ledger
"Alice" -> Get Revocation Registry Definition from Ledger
"Alice" -> Get Revocation Registry Delta from Ledger
%s - Create Revocation State Alice
"Alice" -> Create "Loan-Application-Basic" Proof
"Alice" -> Send "Loan-Application-Basic" Proof to Thrift
"Thrift" -> Get Schemas, Credential Definitions and Revocation Registries from Ledger required for Proof verifying
"Thrift" -> Get Schema from Ledger
"Thrift" -> Get Claim Definition from Ledger
"Thrift" -> Get Revocation Definition from Ledger
"Thrift" -> Get Revocation Registry 

In [58]:
print(" \"Sovrin Steward\" -> Close and Delete wallet")
await wallet.close_wallet(steward['wallet'])
await wallet.delete_wallet(steward['wallet_config'], steward['wallet_credentials'])

print("\"Government\" -> Close and Delete wallet")
await wallet.close_wallet(government['wallet'])
await wallet.delete_wallet(wallet_config("delete", government['wallet_config']),
                           wallet_credentials("delete", government['wallet_credentials']))

print("\"Faber\" -> Close and Delete wallet")
await wallet.close_wallet(faber['wallet'])
await wallet.delete_wallet(wallet_config("delete", faber['wallet_config']),
                           wallet_credentials("delete", faber['wallet_credentials']))

print("\"Acme\" -> Close and Delete wallet")
await wallet.close_wallet(acme['wallet'])
await wallet.delete_wallet(wallet_config("delete", acme['wallet_config']),
                           wallet_credentials("delete", acme['wallet_credentials']))

print("\"Thrift\" -> Close and Delete wallet")
await wallet.close_wallet(thrift['wallet'])
await wallet.delete_wallet(wallet_config("delete", thrift['wallet_config']),
                           wallet_credentials("delete", thrift['wallet_credentials']))

print("\"Alice\" -> Close and Delete wallet")
await wallet.close_wallet(alice['wallet'])
await wallet.delete_wallet(wallet_config("delete", alice['wallet_config']),
                           wallet_credentials("delete", alice['wallet_credentials']))

print("Close and Delete pool")
await pool.close_pool_ledger(pool_['handle'])
await pool.delete_pool_ledger_config(pool_['name'])

print("Getting started -> done")

 "Sovrin Steward" -> Close and Delete wallet
"Government" -> Close and Delete wallet
"Faber" -> Close and Delete wallet
"Acme" -> Close and Delete wallet
"Thrift" -> Close and Delete wallet
"Alice" -> Close and Delete wallet
Close and Delete pool
Getting started -> done


# END